
# 🗽 City 311 Insights Challenge — NYC Edition

## 📍 0 · Executive Scenario

New York City’s 311 hotline handles **tens of thousands of non-emergency service requests daily**, from pothole reports and noise complaints to broken streetlights and missed trash pickups.

The **Mayor’s Office** wants to shift from reactive responses to **proactive service delivery**.

Your data science team must build a **turnkey solution** that:

* 🔮 **Forecasts total daily 311 call volume** to optimize crew scheduling and overtime planning.
* 🚨 **Flags extreme spikes or dips** (due to holidays, storms, parades, or data glitches) so supervisors can take immediate action.
* 🗺️ **Visualizes hot-spots on an interactive NYC map**, with additional insights to help non-technical officials explore trends intuitively.

This solution will influence **daily staffing**, **budget planning**, and **public communication decisions** for **9 million New Yorkers**.

---

## 📌 1 · Problem Statement

### ✅ You must deliver:

#### 1. Daily Volume Forecast

* Predict **total daily 311 calls** for every day in the **evaluation window**.

#### 2. Anomaly Detection

* Identify and rank the **Top-5 largest spikes** and **Top-5 deepest dips** (by percent deviation from forecast).
* Optionally add a short **note** for known causes (e.g., Snowstorm, Holiday).

#### 3. Interactive Dashboard

* Must include:

  1. 📍 **NYC map** showing request density
  2. 📊 At least **3 extra visualizations**
  3. 🎛️ Filters: date range, borough, etc.
  4. 🧭 Clear legends, tooltips, responsive layout

> The dashboard should be fully **usable by non-technical stakeholders**.

---

## 📆 2 · Data Windows

| Purpose              | Period                      | Usage Rules                                                                                                                                     |
| -------------------- | --------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------- |
| Train & Feature Eng. | `2024-08-01` → `2025-04-30` | Only use data in this window for modeling. External data allowed **only** if dated ≤ `2025-04-30` and placed in `data/additional_features.csv`. |
| Forecast Horizon     | `2025-05-01` → `2025-08-01` | Predict one row per day; true labels hidden until scoring.                                                                                      |

* 🗃️ Raw Data: [NYC Open Data — 311 Service Requests](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data)

---

## 📁 3 · Deliverables & GitHub Structure

> ⚠️ Do **NOT** push `train.csv` and `test.csv` to the repo.

### Required Output Files

1. **submission.csv**

| Column                  | Description                                   |
| ----------------------- | --------------------------------------------- |
| `Date`                  | Format: `YYYY-MM-DD`                          |
| `Predicted_Total_Calls` | Integer or float (evaluation can cast to int) |

2. **anomalies.csv**

| Column          | Description                                           |
| --------------- | ----------------------------------------------------- |
| `Date`          | Format: `YYYY-MM-DD`                                  |
| `Actual`        | Observed call volume (integer)                        |
| `Expected`      | Model's forecasted value (float)                      |
| `Anomaly_Score` | % deviation or difference (document method in README) |
| `Note`          | Optional note for context (e.g., Holiday, Storm)      |

### Submission Rules

* `submission.ipynb` must run **end-to-end** on a **fresh CPU-only** clone within **≤10 minutes**
* It must generate both `submission.csv` and `anomalies.csv` in the repo **root**
* 🧪 **Reproducibility is critical**—no points for unclear or broken code

### README

Must include:

* 📋 Short project description
* 📦 Instructions to run the notebook
* 🔧 Details of modeling & anomaly scoring method
* 🧪 Two **copy-paste** commands:

  * One to set up the environment
  * One to run the notebook

### Bonus

* 🌐 If dashboard is deployed (e.g., Streamlit, Vercel), include a public URL in `dashboard_url.txt`
  **➕ Worth +5 bonus points!**

---

## 🧮 4 · Tasks & Scoring (Total: 100 pts + 5 bonus)

| #  | Task              | Criteria                                                      | Points |
| -- | ----------------- | ------------------------------------------------------------- | ------ |
| T1 | Data Preparation  | Cleaning, feature design, clear justification                 | 10     |
| T2 | Forecast Model    | RMSE / MAPE on hidden forecast window                         | 30     |
| T3 | Anomaly Detection | Precision / F1 on spikes and dips                             | 10     |
| T4 | Dashboard         | NYC map, extra visuals, good UX, interactivity                | 30     |
| T5 | Report & Code     | ≤ 2-page insight report, 3-min video, clean reproducible repo | 20     |
| —  | **Bonus**         | Public online dashboard link                                  | **+5** |

---

## 🧪 5 · Technical Guidelines

* Python version: `3.10`
* **No pre-trained LLMs** allowed for direct prediction
* Make the GitHub repo **public**
* Prioritize skills in:

  * ✅ Time-Series Forecasting
  * ⚠️ Anomaly Detection
  * 📊 Interactive Visualization (e.g., Plotly, Folium, Dash, Streamlit)




-  We should convert the hourly, minute 311 calls into daily in order to get target column

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv("../data/nyc_311_filtered.csv")
df.head(2)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,62291042,2024-09-01T00:00:00.000,2024-09-01T00:20:58.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11413.0,135-41 223 STREET,223 STREET,135 AVENUE,137 AVENUE,135 AVENUE,137 AVENUE,ADDRESS,SPRINGFIELD GARDENS,223 STREET,NaN,Closed,NaN,The Police Department responded to the complai...,2024-09-01T00:21:02.000,13 QUEENS,4.131180e+09,QUEENS,1054165.0,185440.0,MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.675391,-73.747948,"\n, \n(40.6753906302629, -73.74794760083083)"
1,62291351,2024-08-31T23:59:55.000,2024-09-01T00:40:53.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10461.0,1448 GILLESPIE AVENUE,GILLESPIE AVENUE,HARRINGTON AVENUE,DUDLEY AVENUE,HARRINGTON AVENUE,DUDLEY AVENUE,ADDRESS,BRONX,GILLESPIE AVENUE,NaN,Closed,NaN,The Police Department responded to the complai...,2024-09-01T00:40:57.000,10 BRONX,NaN,BRONX,1031042.0,245942.0,MOBILE,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.841605,-73.830888,"\n, \n(40.8416052035878, -73.83088758672871)"


In [10]:
x = df.head(10)
x.to_csv("x.csv")

### Data handling

In [3]:
#Make a copy to avoid modifying original data
data = df.copy()

# Convert date column to datetime if it's not already
data['created_date'] = pd.to_datetime(data['created_date'])

# Extract date only (remove time component)
data['date'] = data['created_date'].dt.date

# Basic daily aggregation - total calls per day
daily_calls = data.groupby('date').size().reset_index(name='total_calls')

# Convert date back to datetime for easier manipulation
daily_calls['date'] = pd.to_datetime(daily_calls['date'])

In [4]:
# Clean borough names
data['borough'] = data['borough'].str.upper().str.strip()
borough_mapping = {
    'MANHATTAN': 'Manhattan',
    'BROOKLYN': 'Brooklyn', 
    'QUEENS': 'Queens',
    'BRONX': 'Bronx',
    'STATEN ISLAND': 'Staten Island'
}
data['borough'] = data['borough'].map(borough_mapping).fillna('Unknown')
print(f"Cleaned boroughs: {data['borough'].value_counts()}")

Cleaned boroughs: borough
Brooklyn         1001567
Queens            804198
Bronx             760983
Manhattan         681527
Staten Island     122153
Unknown             2626
Name: count, dtype: int64


#### Data Cleaning

In [5]:
# drop duplicates
print(data.shape)
data = data.drop_duplicates()
print(data.shape)

(3373054, 42)
(3373037, 42)


In [6]:
# Calculate null counts and percentages
null_counts = data.isnull().sum()
null_percent = (null_counts / len(data)) * 100

null_report = pd.DataFrame({
    'Null Count': null_counts,
    'Null Percentage': null_percent.round(2)
})

# Print nulls info
print("Null Values Report:\n")
print(null_report[null_report['Null Count'] > 0])

# Drop columns with more than X% nulls
null_threshold = 3.5  # Change this as needed
cols_to_drop = null_report[null_report['Null Percentage'] > null_threshold].index

data = data.drop(columns=cols_to_drop)

print(f"\nDropped columns with > {null_threshold}% nulls: {list(cols_to_drop)}")
print(f"Remaining columns: {data.shape[1]}")


# Remove high missing value columns
columns_to_remove = [
    'location',
    "resolution_action_updated_date", 
    "resolution_description",
    "street_name",
    "status",
    "agency",
    "unique_key"
]

# Remove columns
data = data.drop(columns=columns_to_remove, errors='ignore')
print(f"Cleaned shape: {data.shape}")
print(f"Removed {len(columns_to_remove)} columns")

Null Values Report:

                                Null Count  Null Percentage
closed_date                         123694             3.67
descriptor                           99427             2.95
location_type                       414233            12.28
incident_zip                         30617             0.91
incident_address                    117890             3.50
street_name                         118005             3.50
cross_street_1                      890882            26.41
cross_street_2                      890200            26.39
intersection_street_1              1034358            30.67
intersection_street_2              1032773            30.62
address_type                         13699             0.41
city                                151176             4.48
landmark                           1266485            37.55
facility_type                      3360718            99.63
due_date                           3355254            99.47
resolution_descript

In [7]:
# Convert to datetime
daily_calls['date'] = pd.to_datetime(daily_calls['date'])

# Sort by datetime before resampling
daily_calls = daily_calls.sort_values('date')
daily_calls

,date,total_calls
0,2024-08-01,9300
1,2024-08-02,9077
2,2024-08-03,7873
3,2024-08-04,8297
4,2024-08-05,10719
...,...,...
346,2025-07-27,9150
347,2025-07-28,9143
348,2025-07-29,9098
349,2025-07-30,8272


In [8]:
daily_df = daily_calls.copy()


### Data analysis

In [9]:
# Graph 1: Daily call volume over time
plt.figure(figsize=(15, 6))
plt.plot(merged_df['date'], merged_df['total_calls'], linewidth=1, alpha=0.8)
plt.title('Daily 311 Call Volume Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Calls', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

NameError: name 'merged_df' is not defined

<Figure size 1500x600 with 0 Axes>

From the chart above, we can observe distinct seasonal trends in NYC 311 call volumes over different times. **Higher activity levels, averaging approximately 10,000-11,000 calls per day, occurred during the fall and winter months (September through January).** Conversely, **lower activity levels, averaging around 8,500-9,500 calls per day, were evident during the spring and summer months (March through July).** A clear seasonal decline is visible from January to July, suggesting that colder months generate more municipal service requests, likely due to weather-related issues, heating problems, and increased indoor activities that lead to more complaints.

In [ ]:
# Graph 2: Day of week pattern
plt.figure(figsize=(10, 6))
dow_avg = merged_df.groupby('dayofweek')['total_calls'].mean()
bars = plt.bar(range(7), dow_avg.values, color='skyblue', alpha=0.8, edgecolor='black')
plt.title('Average 311 Calls by Day of Week', fontsize=16, fontweight='bold')
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Average Calls', fontsize=12)
plt.xticks(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.xticks(rotation=45) 

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

The day-of-week pattern shows weekdays consistently generating more 311 calls than weekends, with Monday and Tuesday being the busiest days (~510,000 calls each). Saturday has the lowest call volume (~440,000 calls), representing about a 15% decrease from peak weekdays, which makes sense as many city services and businesses operate on reduced schedules and fewer people are engaged in activities that generate complaints.

In [ ]:
# Graph 3: Monthly pattern with year
plt.figure(figsize=(15, 6))

# Create year-month combination for grouping
merged_df['year_month'] = merged_df['date'].dt.to_period('M')
monthly_avg = merged_df.groupby('year_month')['total_calls'].mean()

# Create x-axis positions
x_positions = range(len(monthly_avg))
bars = plt.bar(x_positions, monthly_avg.values, color='lightcoral', alpha=0.8, edgecolor='black')

plt.title('Average 311 Calls by Month and Year', fontsize=16, fontweight='bold')
plt.xlabel('Month-Year', fontsize=12)
plt.ylabel('Average Calls', fontsize=12)

# Create custom labels (e.g., "Jan 2024", "Feb 2024", etc.)
labels = [str(period) for period in monthly_avg.index]
plt.xticks(x_positions, labels, rotation=45)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

<b>Seasonal Pattern:</b> NYC 311 calls peak in winter (January: 11,232) due to weather-related issues like heating and infrastructure problems, then drop to their lowest in summer (July: 8,517) when fewer emergencies occur. The data shows a clear seasonal cycle with calls gradually increasing through fall/winter and stabilizing in spring/summer months.

In [ ]:
# Graph 4: Seasonal trends by quarter
plt.figure(figsize=(10, 6))
quarterly_avg = merged_df.groupby('quarter')['total_calls'].mean()
bars = plt.bar(quarterly_avg.index, quarterly_avg.values, 
               color=['lightblue', 'lightgreen', 'orange', 'lightcoral'], 
               alpha=0.8, edgecolor='black')

plt.title('Average 311 Calls by Quarter', fontsize=16, fontweight='bold')
plt.xlabel('Quarter', fontsize=12)
plt.ylabel('Average Calls', fontsize=12)
plt.xticks([1, 2, 3, 4], ['Q1\n(Jan-Mar)', 'Q2\n(Apr-Jun)', 'Q3\n(Jul-Sep)', 'Q4\n(Oct-Dec)'])

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Graph 5: Distribution of daily calls
plt.figure(figsize=(12, 6))
plt.hist(merged_df['total_calls'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')

# Add mean and median lines
mean_calls = merged_df['total_calls'].mean()
median_calls = merged_df['total_calls'].median()
plt.axvline(mean_calls, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_calls:.0f}')
plt.axvline(median_calls, color='blue', linestyle='--', linewidth=2, label=f'Median: {median_calls:.0f}')

plt.title('Distribution of Daily 311 Call Volumes', fontsize=16, fontweight='bold')
plt.xlabel('Daily Calls', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print distribution statistics
print("Distribution Statistics:")
print(f"Mean: {merged_df['total_calls'].mean():.2f}")
print(f"Median: {merged_df['total_calls'].median():.2f}")
print(f"Standard Deviation: {merged_df['total_calls'].std():.2f}")
print(f"Minimum: {merged_df['total_calls'].min()}")
print(f"Maximum: {merged_df['total_calls'].max()}")
print(f"25th Percentile: {merged_df['total_calls'].quantile(0.25):.2f}")
print(f"75th Percentile: {merged_df['total_calls'].quantile(0.75):.2f}")

The daily 311 call volumes follow a roughly bell-shaped distribution centered around 9,000-9,500 calls per day, with the mean (9610) slightly higher than the median (9319), indicating a slight right skew due to occasional high-volume days. Most days fall within the 8,000-11,000 call range, with some outlier days reaching 14,000+ calls and rare low-volume days below 6,000 calls.

In [ ]:
# Graph 6: Box plot for distribution summary
plt.figure(figsize=(12, 6))
plt.boxplot(merged_df['total_calls'], vert=False, patch_artist=True, 
            boxprops=dict(facecolor='lightgreen', alpha=0.7))
plt.title('Box Plot of Daily 311 Call Volume Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Number of Daily Calls', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The box plot reveals that 50% of days fall between roughly 8,500-10,500 calls (the box), with numerous outlier days above 12,000 calls and a few rare low-volume days below 6,000 calls

In [ ]:
# Graph 7: Q-Q plot for normality check
plt.figure(figsize=(10, 6))
stats.probplot(merged_df['total_calls'], dist="norm", plot=plt)
plt.title('Q-Q Plot: Normal Distribution Check', fontsize=16, fontweight='bold')
plt.xlabel('Theoretical Quantiles', fontsize=12)
plt.ylabel('Ordered Values', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The points curve away from the red diagonal line at both ends, confirming the data is not perfectly normally distributed - it has heavier tails than a normal distribution, especially on the high end.

In [ ]:
# Graph 8: Cumulative distribution function
plt.figure(figsize=(12, 6))
sorted_calls = np.sort(merged_df['total_calls'])
y = np.arange(1, len(sorted_calls) + 1) / len(sorted_calls)
plt.plot(sorted_calls, y, marker='.', linestyle='none', alpha=0.7, color='steelblue')
plt.title('Cumulative Distribution of Daily 311 Calls', fontsize=16, fontweight='bold')
plt.xlabel('Number of Daily Calls', fontsize=12)
plt.ylabel('Cumulative Probability', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The S-shaped curve shows that 80% of days have fewer than 11,000 calls, with a steep rise between 7,000-11,000 calls where most daily volumes occur, then a gradual tail for the remaining high-volume days.

In [ ]:
# Print comprehensive distribution statistics
print("=" * 50)
print("DISTRIBUTION STATISTICS SUMMARY")
print("=" * 50)
print(f"Mean: {merged_df['total_calls'].mean():.2f}")
print(f"Median: {merged_df['total_calls'].median():.2f}")
print(f"Standard Deviation: {merged_df['total_calls'].std():.2f}")
print(f"Minimum: {merged_df['total_calls'].min()}")
print(f"Maximum: {merged_df['total_calls'].max()}")
print(f"Range: {merged_df['total_calls'].max() - merged_df['total_calls'].min()}")
print("-" * 50)
print("PERCENTILES:")
print(f"25th Percentile (Q1): {merged_df['total_calls'].quantile(0.25):.2f}")
print(f"50th Percentile (Q2/Median): {merged_df['total_calls'].quantile(0.50):.2f}")
print(f"75th Percentile (Q3): {merged_df['total_calls'].quantile(0.75):.2f}")
print(f"90th Percentile: {merged_df['total_calls'].quantile(0.90):.2f}")
print(f"95th Percentile: {merged_df['total_calls'].quantile(0.95):.2f}")
print(f"99th Percentile: {merged_df['total_calls'].quantile(0.99):.2f}")
print("-" * 50)
print("DISTRIBUTION SHAPE:")
print(f"Skewness: {merged_df['total_calls'].skew():.2f}")
print(f"Kurtosis: {merged_df['total_calls'].kurtosis():.2f}")
print("-" * 50)
print("OUTLIER DETECTION (IQR Method):")
Q1 = merged_df['total_calls'].quantile(0.25)
Q3 = merged_df['total_calls'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = merged_df[(merged_df['total_calls'] < lower_bound) | (merged_df['total_calls'] > upper_bound)]
print(f"IQR: {IQR:.2f}")
print(f"Lower Bound: {lower_bound:.2f}")
print(f"Upper Bound: {upper_bound:.2f}")
print(f"Number of Outliers: {len(outliers)}")
print(f"Percentage of Outliers: {(len(outliers)/len(merged_df))*100:.2f}%")
print("=" * 50)